In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -q transformers
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q datasets --upgrade
!pip install -q accelerate
!pip install -q sentencepiece
!pip install -q torch
!pip install -q numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import json
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os # Для создания директории
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os # Для проверки пути
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import os

In [6]:
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU доступен: {gpu_name}")
    device = torch.device("cuda")
else:
    print("GPU не доступен")
    device = torch.device("cpu")

GPU доступен: Tesla T4


# **Загрузка модели и токенизатора (с 4-битным квантованием)**

In [7]:
model_name = "Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning"
# model_name = "sberbank-ai/rugpt3medium_based_on_gpt2" - как показала практика буду использовать эту модель. но не суть

# Конфигурация для 4-битного квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # Тип квантования - это база (часто используют nf4)
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True, # Использовать двойное квантование для еще большей экономии, экономим максимально
)

# Загрузка модели с квантованием
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True #
    )
    print(f"Модель '{model_name}' успешно загружена с 4-битным квантованием ураааа")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")
    print("Попробуем загрузить без явного torch_dtype=torch.bfloat16, если ошибка связана с ним")


###################################___Еще_одна_вариация_###########################################

    # Иногда на T4 bfloat16 может вызывать проблемы, попробуем с float16, поэтому попробуем след финт ушами
    bnb_config_fp16 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16, # изменение только тут
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config_fp16,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    print(f"Модель '{model_name}' успешно загружена с 4-битным квантованием и compute_dtype=torch.float16.")



######################################__________ТОКИНИЗАТОР___________###################################


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Настройка токенизатора GPT-2 модели часто не имеют специального pad_token
# Мы можем использовать eos_token (end-of-sequence) в качестве pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Установлен pad_token: {tokenizer.pad_token} (был None, использован eos_token)")

# И уже по цепочке надо обновим конфигурацию модели, если pad_token_id не был установлен
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id
    print(f"Установлен model.config.pad_token_id: {model.config.pad_token_id}")


############__Я_стараюсь_написать_ну_прям_очень_наглядный код,_поэтому_буду_выводить_все_в_этом_мире_########

print("\nИнформация о модели:")
print(f"  Количество параметров: {model.num_parameters() / 1e6:.2f}M (после квантования может быть меньше)") # num_parameters может не отражать фактический размер в памяти после квантования
print(f"  Конфигурация модели: {model.config.__class__.__name__}")

print("\nИнформация о токенизаторе:")
print(f"  Класс токенизатора: {tokenizer.__class__.__name__}")
print(f"  Размер словаря: {tokenizer.vocab_size}")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print(f"  BOS token: {tokenizer.bos_token} (ID: {tokenizer.bos_token_id})") # Обычно нет у GPT2

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

Модель 'Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning' успешно загружена с 4-битным квантованием ураааа


...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

Установлен model.config.pad_token_id: 151645

Информация о модели:
  Количество параметров: 1720.57M (после квантования может быть меньше)
  Конфигурация модели: Qwen3Config

Информация о токенизаторе:
  Класс токенизатора: Qwen2TokenizerFast
  Размер словаря: 151643
  Pad token: <|endoftext|> (ID: 151643)
  EOS token: <|im_end|> (ID: 151645)
  BOS token: None (ID: None)


# **Что у нас по данным, Джонии? Загрузка и просмотр данных ₍^. .^₎⟆**

In [8]:
file_path = "/content/drive/MyDrive/output.jsonl" # Путь к вашему файлу, пожалуйста, настройка под себя, укажите свой путь
data = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    print(f"Успешно загружено {len(data)} записей из {file_path}")
except Exception as e:
    print(f"Ошибка при чтении файла {file_path}: {e}")

if data:
    dataset = Dataset.from_list(data)

    print("\nПервые 3 примера из датасета:")
    for i in range(min(3, len(dataset))):
        print(dataset[i])

    # Инфа о датасете
    print("\nИнформация о датасете:")
    print(dataset)
else:
    print("Данные не были загружены. Проверьте путь к файлу и его содержимое и прочитайте самый первый мой комментарий")

Успешно загружено 8641 записей из /content/drive/MyDrive/output.jsonl

Первые 3 примера из датасета:
{'instruction': 'Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?', 'input': '', 'output': 'Карьерный путь, науку и образование.'}
{'instruction': 'Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?', 'input': '', 'output': 'Экономика и физика.'}
{'instruction': 'Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?', 'input': '', 'output': 'Ответ на этот вопрос можно найти в интервью, но в данном тексте он не представлен.'}

Информация о датасете:
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 8641
})


# **Форматирование данных и создание текстовых промптов**

In [9]:
# Проверка существования и заполненности датасета (да, в какой-то момент у меня были с этим проблемы)
if 'dataset' not in globals() or not dataset:
    print("Warning, исходный датасет не был загружен или пуст")
else:
    eos_token = tokenizer.eos_token  # Получаем eos_token из токенизатора
    if eos_token is None:
        print("Предупреждение: eos_token не найден, подумай почему")
        eos_token = ""

    def create_prompt_text(example):
        prompt_template = f"Инструкция: {example['instruction']}\n\nОтвет: {example['output']}{eos_token}"
        return {"text": prompt_template}

    # Применяем функцию ко всему датасету для создания новой колонки 'text'
    dataset_with_text_prompts = dataset.map(
        create_prompt_text,
        remove_columns=['instruction', 'input', 'output']
    )

    print("\nПервые 3 примера сгенерированных текстовых промптов")
    for i in range(min(3, len(dataset_with_text_prompts))):
        print(f"--- Пример {i+1} ---")
        print(dataset_with_text_prompts[i]['text'])
        print("--------------------")

    print("\nИнформация о датасете с текстовыми промптами:")
    print(dataset_with_text_prompts)
    if len(dataset_with_text_prompts) > 0:
        print("\nПример первой записи (ключи):")
        print(dataset_with_text_prompts[0].keys())
        print("\nТекст первой записи:")
        print(dataset_with_text_prompts[0]['text'])
    else:
        print("Датасет с текстовыми промптами пуст, проверяй где накосячил")

Map:   0%|          | 0/8641 [00:00<?, ? examples/s]


Первые 3 примера сгенерированных текстовых промптов
--- Пример 1 ---
Инструкция: Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?

Ответ: Карьерный путь, науку и образование.<|im_end|>
--------------------
--- Пример 2 ---
Инструкция: Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?

Ответ: Экономика и физика.<|im_end|>
--------------------
--- Пример 3 ---
Инструкция: Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?

Ответ: Ответ на этот вопрос можно найти в интервью, но в данном тексте он не представлен.<|im_end|>
--------------------

Информация о датасете с текстовыми промптами:
Dataset({
    features: ['text'],
    num_rows: 8641
})

Пример первой записи (ключи):
dict_keys(['text'])

Текст первой записи:
Инструкция: Какую тему в интервью с Балакиной Татьяной Петровной обсудили в первую очередь?

Ответ: Карьерный путь, науку и образование.<|im_end|>


# **Поработаем с токенизацией, далее поделим на train/val + установим формат**

In [10]:
#########################_Проверка_мол_все_ли_у_нас_хорошо_#########################

if 'dataset_with_text_prompts' not in globals() or not dataset_with_text_prompts:
    print("Ошибка: 'dataset_with_text_prompts' не найден, ищи ошибку")
else:
    def tokenize_function(examples):

        # Пояснительная бригада, что я натыкал и для чего

        # max_length можно будет подобрать позже, пока оставим значение по умолчанию токенизатора
        # или установим достаточно большое, чтобы вместить большинство наших промптов, но пока 512
        # truncation=True обрежет слишком длинные последовательности
        # padding="max_length" добавит паддинг до max_length


        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

    # Токенизируем тексты
    tokenized_dataset_full = dataset_with_text_prompts.map(tokenize_function, batched=True, remove_columns=["text"])
    print("\nИнформация о полном токенизированном датасете до разделения на train/val")
    print(tokenized_dataset_full)

#########################_Делим_данные_на_train/val_#########################
    try:
        split_dataset = tokenized_dataset_full.train_test_split(test_size=0.1, seed=42)
        train_dataset = split_dataset['train']
        eval_dataset = split_dataset['test']
        train_dataset.set_format("torch")
        eval_dataset.set_format("torch")

        print("\nДатасет успешно разделен train/val")
        print(f"Размер train: {len(train_dataset)}")
        print(f"Размер val: {len(eval_dataset)}")


    except Exception as e:
        print(f"Произошла ошибка, иди исправляй: {e}")

Map:   0%|          | 0/8641 [00:00<?, ? examples/s]


Информация о полном токенизированном датасете до разделения на train/val
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 8641
})

Датасет успешно разделен train/val
Размер train: 7776
Размер val: 865


# **Конфигурация LoRA и QLoRA**

# **Подготовка модели к PEFT и конфигурация LoRA**

In [11]:
if 'model' not in globals() or 'tokenizer' not in globals():
    print("Warning: Модель или токенизатор не загружены, ищи ошибку")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("Warning: Обучающий или валидационный датасет не готовы")
else:
    print("Модель, токенизатор и датасеты готовы")

# Подготовка модели для k-битного обучения (QLoRA)
try:
    model = prepare_model_for_kbit_training(model)
    print("Модель подготовлена для k-bit training QLoRA")
except Exception as e:
    print(f"Ошибка при prepare_model_for_kbit_training: {e}")
    print("Продолжаем без этого шага, но могут быть проблемы со стабильностью QLoRA, так что думайте")

# Конфигурация LoRA для модели Qwen-based
lora_config = LoraConfig(
    r=32,  # Увеличили ранг до 32 для большей гибкости
    lora_alpha=64,  # Соответственно, alpha = 2 * r
    target_modules=[
        "q_proj",  # Query projection
        "k_proj",  # Key projection
        "v_proj",  # Value projection
        # Можно раскомментировать для адаптации большего количества слоёв
        # "o_proj",
        # "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
print("\nLoraConfig создана:")
print(lora_config)

# Применение LoRA к модели
try:
    lora_model = get_peft_model(model, lora_config)
    print("\nLoRA применена, красава")
    print("\nИнформация о параметрах после применения LoRA")
    lora_model.print_trainable_parameters()
    model_to_train = lora_model
except Exception as e:
    print(f"Ошибка при применении LoRA к модели: {e}")
    model_to_train = None

# Проверка готовности модели
if model_to_train:
    print("\nМодель готова к обучению с LoRA")
else:
    print("\nМодель НЕ готова к обучению, чек ошибки выше")

Модель, токенизатор и датасеты готовы
Модель подготовлена для k-bit training QLoRA

LoraConfig создана:
LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=32, target_modules={'k_proj', 'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

LoRA применена, красава

Информация о параметрах после применения LoRA
trainable params: 9,175,040 || all params: 1,729,750,016 || trainable%: 0.5304

Модель готова к обучению с LoRA


# **Настройка аргументов обучения + запуск Trainer**

In [12]:
# Проверка готовности компонентов
if 'model_to_train' not in globals() or not model_to_train:
    print("Warning: Модель для обучения model_to_train не определена, ищи ошибку")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("Warning: Обучающий или валидационный датасет не готовы")
elif 'tokenizer' not in globals():
    print("Warning: Токенизатор не загружен")
else:
    print("Все компоненты для обучения готовы, красава")

    # Определяем директорию для сохранения результатов
    output_dir = "./results_qvikhr_lora"

    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir="./results_qvikhr_lora_epoch1_lr5e-5",
        per_device_train_batch_size=2,  # Уменьшено для крупной модели
        gradient_accumulation_steps=8,  # Увеличено для компенсации
        num_train_epochs=1,
        learning_rate=5e-5,  # Скорректировано для крупной модели
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=100,
        eval_strategy="steps",
        eval_steps=100,
        do_eval=True,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        report_to="none",
        fp16=True,
    )

    # Формирование батчей данных
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Создание Trainer и запуск обучения
    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\nTrainer сконфигурирован, начинаем обучение")

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")

        final_model_path = f"{output_dir}/final_model_adapters"
        trainer.save_model(final_model_path)
        print(f"Финальные LoRA адаптеры сохранены в: {final_model_path}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-12-014f1ee62eea>:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения готовы, красава

Trainer сконфигурирован, начинаем обучение


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,1.669000,1.591750
200,1.521700,1.518538
300,1.486500,1.486942
400,1.458500,1.468172


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/


Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_qvikhr_lora/final_model_adapters

Метрики обучения:
{'train_runtime': 5080.2337, 'train_samples_per_second': 1.531, 'train_steps_per_second': 0.096, 'total_flos': 3.3886979003252736e+16, 'train_loss': 1.5558262656255024, 'epoch': 1.0}

Лог обучения (история):
{'loss': 2.0019, 'grad_norm': 0.6489819288253784, 'learning_rate': 4.495884773662551e-05, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 1.669, 'grad_norm': 0.9473133087158203, 'learning_rate': 3.981481481481482e-05, 'epoch': 0.205761316872428, 'step': 100}
{'eval_loss': 1.5917502641677856, 'eval_runtime': 130.9156, 'eval_samples_per_second': 6.607, 'eval_steps_per_second': 0.833, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 1.5483, 'grad_norm': 0.9301230907440186, 'learning_rate': 3.467078189300412e-05, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 1.5217, 'grad_norm': 0.9450205564498901, 'learning_rate': 2.952674897119342e-05, 'epoch': 0.41152263374

# **Тестирование Дообученной Модели в Colab**

In [17]:


# Устанавливаем model_name
model_name = "Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning"

# Проверка и загрузка токенизатора
if 'tokenizer' not in globals() or tokenizer.name_or_path != model_name:
    print(f"Загрузка токенизатора для {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token}")

# Путь к сохраненным LoRA адаптерам
lora_adapters_path = "/content/results_qvikhr_lora/final_model_adapters"

# Загрузка конфигурации PEFT LoRA
try:
    config = PeftConfig.from_pretrained(lora_adapters_path)
    print("PeftConfig успешно загружен.")
except Exception as e:
    print(f"Ошибка при загрузке PeftConfig: {e}. Проверьте путь: {lora_adapters_path}")
    config = None

# Загрузка базовой модели с квантованием
if config:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )
    print(f"Загрузка базовой модели '{model_name}' с квантованием для инференса")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием, красава")

        # Загрузка модели с LoRA адаптерами
        print(f"Загрузка модели с LoRA адаптерами из {lora_adapters_path}...")
        inference_model = PeftModel.from_pretrained(base_model, lora_adapters_path)
        inference_model.eval()
        final_model_for_inference = inference_model
        print("Модель с LoRA адаптерами готова для инференса, красава")
    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    final_model_for_inference = None

# Функция для генерации ответа
if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, max_new_tokens=100, temperature=0.7, top_p=0.9):
        messages = [{"role": "user", "content": question}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512-max_new_tokens)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)
        return answer_text.strip()

    # Тестовые запросы
    test_questions = [
        "Кто такой Никита Чухров?",
        "Кто такой Филипп Ульянкин?",
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n---###_____Тестирование_модели_____###---")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer)
        print(f"Ответ модели: {answer}")
else:
    print("Модель для инференса не была загружена, ищи ошибки")

PeftConfig успешно загружен.
Загрузка базовой модели 'Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning' с квантованием для инференса
Базовая модель успешно загружена с квантованием, красава
Загрузка модели с LoRA адаптерами из /content/results_qvikhr_lora/final_model_adapters...
Модель с LoRA адаптерами готова для инференса, красава

---###_____Тестирование_модели_____###---

Вопрос: Кто такой Никита Чухров?
Ответ модели: <think>

</think>

Никита Чухров — это российский бизнесмен и предприниматель, известный своими инвестициями в секторы цифровой экономики, включая создание и развитие технологических стартапов. Он был одним из основателей и руководителей компаний, таких как «Сбербанк-Альфа», «МТС», «Сбербанк-Маркет», и других, а также зани

Вопрос: Кто такой Филипп Ульянкин?
Ответ модели: <think>

</think>

Филипп Ульянкин — это российский предприниматель и бизнесмен, известный своими инвестициями в разные сферы экономики, включая технологические, финансовые и строительные проекты. О

In [20]:
#########################_Проверка_компонентов_#########################

if 'model_to_train' not in globals() or not model_to_train:
    print("WARNING: Модель для обучения model_to_train не определена")
    raise RuntimeError("model_to_train не определена, Прерываем принудительно")
elif 'train_dataset' not in globals() or 'eval_dataset' not in globals():
    print("WARNING: train/val датасет не готовы")
    raise RuntimeError("train_dataset или eval_dataset не определены, Прерываем принудительно")
elif 'tokenizer' not in globals():
    print("WARNING: Токенизатор не загружен")
    raise RuntimeError("tokenizer не определен, Прерываем принудительно")
else:
    print("Все компоненты для обучения готовы")

    # Директория для сохранения результатов
    current_output_dir = "./results_qvikhr_lora_epoch1_lr5e-5"  # Пожалуйста, указывайте свой путь
    print(f"Результаты будут сохранены в: {current_output_dir}")


    training_args = TrainingArguments(
      output_dir="./results_qvikhr_lora_epoch1_lr5e-5",
      per_device_train_batch_size=2,          # если GPU позволяет, можно увеличить
      gradient_accumulation_steps=8,            # effective batch size = 16
      num_train_epochs=1,
      learning_rate=5e-5,
      warmup_steps=100,                         # добавили warmup
      max_grad_norm=1.0,                        # градиент-клиппинг для стабильности
      logging_strategy="steps",
     logging_steps=50,                         # можно увеличить при необходимости
      save_strategy="steps",
      save_steps=200,                           # увеличено, чтобы не тормозить обучение
      eval_strategy="steps",
      eval_steps=200,                           # увеличено для экономии времени
      do_eval=True,
      save_total_limit=2,
      load_best_model_at_end=True,
      metric_for_best_model="eval_loss",
      greater_is_better=False,
      report_to="none",
      fp16=True,
    )

    # Data collator для causal language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Проверка, является ли model_to_train PEFT моделью (с LoRA)
    if not hasattr(model_to_train, 'peft_config'):
        print("ПРЕДУПРЕЖДЕНИЕ: 'model_to_train' не выглядит как PEFT модель. Проверьте, что LoRA применена.")
        raise RuntimeError("model_to_train не является PEFT моделью, Прерываем принудительно")

    # Создание Trainer
    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

#########################_Обучение_#########################

    try:
        train_result = trainer.train()
        print("\nОбучение завершено!")
        final_lora_adapters_path = f"{current_output_dir}/final_lora_adapters"
        trainer.save_model(final_lora_adapters_path)
        print(f"Финальные LoRA адаптеры сохранены в: {final_lora_adapters_path}")

        metrics = train_result.metrics
        print("\nМетрики обучения:")
        print(metrics)
        print("\nЛог обучения (история):")
        for log_entry in trainer.state.log_history:
            print(log_entry)

    except Exception as e:
        print(f"\nПроизошла ошибка во время обучения: {e}")
        import traceback
        traceback.print_exc()

<ipython-input-20-55d8434965cb>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Все компоненты для обучения готовы
Результаты будут сохранены в: ./results_qvikhr_lora_epoch1_lr5e-5


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
200,1.397600,1.411917
400,1.372200,1.379972


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



Обучение завершено!
Финальные LoRA адаптеры сохранены в: ./results_qvikhr_lora_epoch1_lr5e-5/final_lora_adapters

Метрики обучения:
{'train_runtime': 4797.886, 'train_samples_per_second': 1.621, 'train_steps_per_second': 0.101, 'total_flos': 3.3886979003252736e+16, 'train_loss': 1.3853268485991552, 'epoch': 1.0}

Лог обучения (история):
{'loss': 1.3899, 'grad_norm': 0.9884285926818848, 'learning_rate': 2.45e-05, 'epoch': 0.102880658436214, 'step': 50}
{'loss': 1.4306, 'grad_norm': 1.3176394701004028, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.205761316872428, 'step': 100}
{'loss': 1.4055, 'grad_norm': 1.217694640159607, 'learning_rate': 4.365284974093264e-05, 'epoch': 0.30864197530864196, 'step': 150}
{'loss': 1.3976, 'grad_norm': 1.2805982828140259, 'learning_rate': 3.717616580310881e-05, 'epoch': 0.411522633744856, 'step': 200}
{'eval_loss': 1.4119168519973755, 'eval_runtime': 130.8153, 'eval_samples_per_second': 6.612, 'eval_steps_per_second': 0.833, 'epoch': 0.41152263374

In [21]:
# Устанавливаем model_name
model_name = "Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning"

# Проверка и загрузка токенизатора
if 'tokenizer' not in globals() or tokenizer.name_or_path != model_name:
    print(f"Загрузка токенизатора для {model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"Установлен pad_token: {tokenizer.pad_token}")

# Путь к сохраненным LoRA адаптерам
lora_adapters_path = "/content/results_qvikhr_lora_epoch1_lr5e-5/final_lora_adapters"

# Загрузка конфигурации PEFT LoRA
try:
    config = PeftConfig.from_pretrained(lora_adapters_path)
    print("PeftConfig успешно загружен.")
except Exception as e:
    print(f"Ошибка при загрузке PeftConfig: {e}. Проверьте путь: {lora_adapters_path}")
    config = None

# Загрузка базовой модели с квантованием
if config:
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )
    print(f"Загрузка базовой модели '{model_name}' с квантованием для инференса")
    try:
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config_inference,
            device_map="auto",
            trust_remote_code=True
        )
        print("Базовая модель успешно загружена с квантованием, красава")

        # Загрузка модели с LoRA адаптерами
        print(f"Загрузка модели с LoRA адаптерами из {lora_adapters_path}...")
        inference_model = PeftModel.from_pretrained(base_model, lora_adapters_path)
        inference_model.eval()
        final_model_for_inference = inference_model
        print("Модель с LoRA адаптерами готова для инференса, красава")
    except Exception as e:
        print(f"Ошибка при загрузке базовой модели или применении LoRA: {e}")
        final_model_for_inference = None
else:
    final_model_for_inference = None

# Функция для генерации ответа
if final_model_for_inference and tokenizer:
    def generate_answer(question, model, tokenizer, max_new_tokens=100, temperature=0.7, top_p=0.9):
        messages = [{"role": "user", "content": question}]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512-max_new_tokens)
        inputs = {k: v.to(next(model.parameters()).device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        answer_tokens = outputs[0][inputs['input_ids'].shape[1]:]
        answer_text = tokenizer.decode(answer_tokens, skip_special_tokens=True)
        return answer_text.strip()

    # Тестовые запросы
    test_questions = [
        "Кто такой Никита Чухров?",
        "Кто такой Филипп Ульянкин?",
        "Кто по образованию Балакина Татьяна Петровна?",
        "Какие направления в образовании упомянуты в контексте обсуждения с Татьяной Петровной?",
        "Почему важно участвовать в олимпиадах и хакатонах по мнению Татьяны Петровны?"
    ]

    print("\n---###_____Тестирование_модели_____###---")
    for q in test_questions:
        print(f"\nВопрос: {q}")
        answer = generate_answer(q, final_model_for_inference, tokenizer)
        print(f"Ответ модели: {answer}")
else:
    print("Модель для инференса не была загружена, ищи ошибки")

PeftConfig успешно загружен.
Загрузка базовой модели 'Vikhrmodels/QVikhr-3-1.7B-Instruction-noreasoning' с квантованием для инференса
Базовая модель успешно загружена с квантованием, красава
Загрузка модели с LoRA адаптерами из /content/results_qvikhr_lora_epoch1_lr5e-5/final_lora_adapters...
Модель с LoRA адаптерами готова для инференса, красава

---###_____Тестирование_модели_____###---

Вопрос: Кто такой Никита Чухров?
Ответ модели: <think>

</think>

Никита Чухров — это человек, чьи интересы и карьера связаны с технологиями, включая работу в Microsoft и развитие продуктов в области анализа данных.

Вопрос: Кто такой Филипп Ульянкин?
Ответ модели: <think>

</think>

Филипп Ульянкин — это один из самых известных и популярных абитуриентов на сайте «Математика и физика». Он является одним из первых, кто увидел на сайте тексты о математике, и его история стала основой для множества других абитуриентов, которые начали писать аналогичные тексты. Он также известен тем, что

Вопрос: Кто по 